In [ ]:
#importing the Libraies
import numpy as np
import pandas as pd

In [ ]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [ ]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [ ]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [ ]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [ ]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [ ]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

# Define parameter grid (smaller for speed)
param_grid = {
   'n_estimators': [50, 100, 200],       # Number of trees
    'max_depth': [None, 5, 10, 20],       # Depth of trees
    'min_samples_split': [2, 5, 10],      # Min samples to split node
    'min_samples_leaf': [1, 2, 5],        # Min samples in leaf
    'bootstrap': [True, False]            # Whether bootstrap samples are used
}


grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X,y)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 5, 10, 20],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='f1_weighted', verbose=3)

In [ ]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [ ]:
# Evaluation
from sklearn.metrics import confusion_matrix,classification_report,f1_score
cm = confusion_matrix(y, y_pred)
clf_report =classification_report(y, y_pred)
f1_macro =f1_score(y, y_pred, average='weighted')

In [ ]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 200}
The f1_macro: 0.9055415288645102
The confusion Matrix:
 [[234  23]
 [ 15 128]]
The report:
               precision    recall  f1-score   support

           0       0.94      0.91      0.92       257
           1       0.85      0.90      0.87       143

    accuracy                           0.91       400
   macro avg       0.89      0.90      0.90       400
weighted avg       0.91      0.91      0.91       400



In [ ]:
# Save Best Model
import pickle
filename = "RandomForestClassifier_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [ ]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.245947,0.004737,0.017935,0.005352,True,None,1,2,50,"{'bootstrap': True, 'max_depth': None, 'min_sa...",0.783072,0.925000,0.859227,0.840551,0.826167,0.846803,0.046467,175
1,0.421020,0.059465,0.034678,0.009057,True,None,1,2,100,"{'bootstrap': True, 'max_depth': None, 'min_sa...",0.767183,0.950000,0.872729,0.840551,0.802370,0.846567,0.062728,176
2,0.553055,0.046696,0.030212,0.001266,True,None,1,2,200,"{'bootstrap': True, 'max_depth': None, 'min_sa...",0.783072,0.937229,0.872729,0.840551,0.814270,0.849570,0.052858,163
3,0.134960,0.002632,0.011298,0.000264,True,None,1,5,50,"{'bootstrap': True, 'max_depth': None, 'min_sa...",0.783072,0.937229,0.872729,0.828357,0.851000,0.854477,0.050939,153
4,0.255217,0.003144,0.017883,0.001539,True,None,1,5,100,"{'bootstrap': True, 'max_depth': None, 'min_sa...",0.798551,0.950000,0.899176,0.828357,0.851000,0.865417,0.053566,103
